In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.dates as md
import joblib
import plotly.express as px
import os
import scipy
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from scipy import signal
from keras.models import load_model
from matplotlib import pyplot as plt

### Defining global

In [ ]:
station_location = "Betong"
directory = os.path.dirname(os.getcwd()) + '/dataset/' + station_location 
dataStorage = os.path.dirname(os.getcwd()) + "/train/" + station_location + "/data/"
testYear = "_" + str(2016)
trainYear = "_" + str(2017)
scaler_filename = dataStorage + "scaler_data_" + station_location + trainYear
model_name = dataStorage + station_location + trainYear + ".h5"

### Defining functions

In [ ]:
def plot_lossfunction(history):
    fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
    ax.plot(history['loss'], 'b', label='Train', linewidth=2)
    ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
    ax.set_title('Model loss', fontsize=16)
    ax.set_ylabel('Loss (mae)')
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper right')
    plt.show()

In [ ]:
def plot_fft(data, title):
    fig,ax = plt.subplots(figsize=(14, 6), dpi=80)
    ax.plot(data[:,0].real, label='FFT', color='green', animated=True, linewidth=1)
    plt.legend(loc='lower left')
    ax.set_title(title, fontsize=16)
    plt.show()

In [ ]:
def format_data(filename, water_name, time_name):

    '''
    - read in the timestamp and waterlevel;
    - select those waterlevel!=nan
    - drop duplicates timestamps & waterlevel and keep the last
    - set the 'timestamp' column into DatetimeIndex and set as index and sort it (timestamp must be monotronic)
    '''
    df = pd.read_csv(filename, usecols=[time_name, water_name])
    df_new = df[df[water_name].notna()]
    print("after droppping na: " + str(df_new.shape))

    # there are duplicates timestamp in the files,keep the last
    df_new = df_new.drop_duplicates(subset=time_name, keep='last', ignore_index=True)
    print("after droppping duplicates: " + str(df_new.shape))

    df_new[time_name] = pd.DatetimeIndex(df_new[time_name],dayfirst=True)
    df_new = df_new.set_index(time_name)
    df_new = df_new.sort_index()
    print("original size: "+str(df.shape))
    print("after sort index: " + str(df_new.shape))
    
    ''''
    - change timestamp from "date" format to "string format" 
    '''
    timestamp = df_new[water_name].index.strftime("%D-M%-Y%")
    waterlevel = df_new[water_name].values
    print(timestamp.shape)
#     plotOriGraph(df_new,timestamp,waterlevel,None,"Original")
    
    return df_new


In [ ]:
def fillWithLine(y, spiketnt, timestamp, waterlevel):
    df_temp = pd.DataFrame()
    df_temp['timestamp'] = timestamp
    df_temp['waterlevel'] = waterlevel

    df_raw = df_temp['waterlevel']
    df_keep = df_raw.loc[np.where(spiketnt!=1)[0]] #find the normal ones
    df_out = pd.merge(df_keep,df_raw,how='outer',left_index=True,right_index=True)

    # Keep only the first column
    s = df_out.iloc[:, 0]

    # 8. Fill missing values
    df_complete = s.fillna(axis=0, method='ffill').fillna(axis=0,method="bfill")
    df_interpolate = s.interpolate()
    df_temp['waterlevel'] = df_complete.values
    df_temp['inter_waterlevel'] = df_interpolate

    return df_temp['waterlevel'].values,df_temp['inter_waterlevel'].values

In [ ]:
def saveToExcelFile(df, time_name, water_name, filename):
#     check directory accuracy
    directory = '../src/train/' + station_location + '/result_lstm/'
    filename = directory + filename + "_result.csv"
    
    if not os.path.exists( directory):
        os.makedirs( directory)
        
    df = df.rename_axis("timestamp")
    df = df.rename(
        columns={
            time_name:"timestamp",
            water_name:"waterlevel"
        })
    df.to_csv(filename)

In [ ]:
def plotOriGraph(df_new, timestamp, waterlevel, waterlevel_flat, title):
    fig = (px.scatter(x = timestamp,y = waterlevel).update_traces(mode='markers+lines'))
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(
        {
            "title":title,
            "xaxis":{
                "title":"timestamp"
            },
            "yaxis":{
                "title":"waterlevel"
            }
        })
    fig.show()

In [ ]:
def plotGraph(df_new, timestamp, waterlevel, waterlevel_flat, title):
    fig = go.Figure()
    fig.add_trace(go.Scattergl(x=timestamp, y=waterlevel,
                    mode='lines+markers',
                    name='Original'))
    fig.add_trace(go.Scattergl(x=timestamp, y=waterlevel_flat,
                    mode='lines+markers',
                    name=title))
#     fig = px.add_line(x=timestamp,y=waterlevel_flat)
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(
        {
            "title":title,
            "xaxis":{
                "title":"timestamp"
            },
            "yaxis":{
                "title":"waterlevel"
            }
        })
    fig.show()

In [ ]:
def spikeWithThreshold(df_waterlevel,TP = 'T'):
        if TP == "T" or TP == "t":
            threshold = 0.6

        elif TP == "NT" or TP == "Nt" or TP == "nt":
            threshold = 0.3
        # time = np.array(df['x'])
        value = np.array(df_waterlevel)
        diff_list = []
        anolist = []
        threshold1 = threshold
        threshold2 =threshold*-1
        anoboo = abs(value) > abs(value) + threshold1 # default all entities in the array to false
    
        for i in range (1, len(value)):         
            diff_list.append(value[i] - value[i-1])
            
        for i in range (0, len(diff_list)):                      
            if diff_list[i] >= threshold1 or diff_list[i] <= threshold2:
                anolist.append(df_waterlevel.index[i+1])
                anoboo[i+1] = True # set to true if spike detected (difference > threshold)
                
        anono = anoboo.copy()
        
        # note : index of anoboo[i] = diff_list[i-1]
        for i in range (0, len(anoboo)):
            if (i != 0) and (i+1 < len(anoboo)):
                if anoboo[i] == True and anoboo[i-1] == True:
                    # if i spike up and i+1 spike down, then i+1 is not a spike
                    # eg : i-1 = 0.5, i = 2.3, i+1 = 0.6, i is spike, i+1 is not a spike
                    if (diff_list[i-1] > 0 and diff_list[i-2] < 0) or (diff_list[i-1] < 0 and diff_list[i-2] > 0):
                        anoboo[i] = False
                        
                    # if i spike up and i+1 spike another up (difference between [(i and i+1) > 0.6] and [(i-1 and i+1 > 1.2)])
                    # eg: i-1 = 0.1, i = 0.73 (>0.6), i+1 = 1.5 (>0.6), so i is not a spike, i+1 is spike
                    elif (diff_list[i-1] > 0 and diff_list[i-2] > 0) or (diff_list[i-1] < 0 and diff_list[i-2] < 0):
                        anoboo[i-1] = False
                
                # if i is spike and i+1 is within the range of 0.59 with i (i+1 = i +- threshold), i is not a spike
                # eg : i-1 = 0.6, i = 4.5, i+1 = 4.6, i is not a spike, i and i+1 is a trend (detect only 1 sharp point spike as spike, else is trend)
                # can write as (abs(diff_list[i-1]) > 0) and (abs(diff_list[i-1]) < threshold1) and ***anoboo[i] == True***:
                elif (abs(diff_list[i-1]) > 0) and (abs(diff_list[i-1]) < threshold1) and (abs(diff_list[i-2]) > threshold1):
                    anoboo[i-1] = False

        return anoboo


In [ ]:
def data(test_filename, scaler_filename, model_name, timestamp_name, waterlevel_name, csv_name, threshold):
    test_data = format_data(test_filename, waterlevel_name, timestamp_name) 
    model = load_model(model_name)
    scaler = joblib.load(scaler_filename)

    # normalize the data
    X_test = scaler.transform(test_data)
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    print("Test data shape:", X_test.shape)

    # calculate the loss on the test set
    X_pred = model.predict(X_test)  
    X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
    X_pred = pd.DataFrame(X_pred, columns=test_data.columns)
    
    X_pred.index = test_data.index
    validation = scaler.inverse_transform(X_pred[waterlevel_name].values.reshape(-1,1))

    scored = pd.DataFrame(index=test_data.index)
    Xtest = X_test.reshape(X_test.shape[0], X_test.shape[2])
    scored['Loss_mae'] =np.mean(np.abs(X_pred - Xtest), axis=1)
    scored['Threshold'] = threshold
    scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
    scored.head()
    
    # Rectify
    print("predicted")
    print(validation.shape)
    # X_pred['actual'] = validation
    test_data['predicted'] = validation
    test_data['loss_mae'] = scored['Loss_mae']
    test_data['anomalies'] = scored['Anomaly']
    test_data['rectified'],test_data['inter'] = fillWithLine(test_data, test_data['anomalies'].values, test_data.index,
                                          test_data[waterlevel_name].values)
    
    
    anomalies = spikewiththreshold(test_data['rectified'],TP="NT")
    test_data['after_checking'],test_data['inter_checking'] = fillWithLine(test_data, anomalies,test_data['rectified'].index, test_data['rectified'].values)
    test_data['median_filter'] = scipy.signal.medfilt(test_data['rectified'], 11)
    test_data['median_inter'] = scipy.signal.medfilt(test_data['inter'], 11)
    
    
    
    saveToExcelFile(test_data,timestamp_name,waterlevel_name,csv_name)
    test_data.head()
    # plotOriGraph(test_data, test_data.index, test_data['rectified'].values, None, title="Rectified")
    # plotGraph(test_data,test_data.index,test_data[waterlevel_name].values,test_data['predicted'].values,title="Predicted")
    plotGraph(test_data,test_data.index,test_data[waterlevel_name].values,test_data['rectified'].values,title="Rectified")
    plotGraph(test_data,test_data.index,test_data[waterlevel_name].values,test_data['after_checking'].values,title="After checking")
    plotGraph(test_data,test_data.index,test_data[waterlevel_name].values,test_data['median_filter'].values,title="Filter")


#### Load and prepare data

In [ ]:
filename = directory + "/" + station_location + testYear + ".csv"

for i in range(2016, 2021):
        filename = directory + "/" + station_location + "_" + str(i) + ".csv"
        print("=========================")
        print("DATA FOR THE YEAR OF", i)
        print("=========================")
        data(filename,
             scaler_filename = scaler_filename,
             model_name = model_name,
             timestamp_name = "timestamp",
             waterlevel_name = "actual_reading",
             csv_name=csv_name,
             threshold=0.7
            )
        